In [ ]:
import asyncio
from util import parse_url
from time import sleep

# twitch IRC server info
SERVER = 'irc.chat.twitch.tv'
CHANNEL = '#...' # set the channel you want to join
NICK = '...' # set your nickname
PASS = 'oauth:token' # you can get it from https://twitchapps.com/tmi/

#'''
from random import random

# freenode irc server example info
SERVER = 'chat.freenode.net:6666'
CHANNEL = '#hojong'
NICK = 'hjbot_%05d' % (random() * 10000)
PASS = None
#'''

class IRCClient(asyncio.Protocol):
    def __init__(self, CHANNEL, NICK, PASS, loop):
        self.channel = CHANNEL
        self.nick = NICK
        self.password = PASS
        self.loop = loop

    def connection_made(self, transport):
        self.transport = transport
        
        msg_list = []
        if self.password and len(self.password) > 0:
            msg_list.append('PASS %s\r\n' % self.password)
        msg_list.append('NICK %s\r\n' % self.nick)
        msg_list.append('USER dummy 0 * :dummy\r\n')
        msg_list.append('JOIN %s\r\n' % self.channel)
        
        for msg in msg_list:
            print('*** [SENDED]', msg, end='')
            transport.write(msg.encode())
            sleep(1)

    def data_received(self, data):
        packet = data.decode().split(' ', 3) # split 3 times
        nickname = (packet[0].split('!'))[0][1:]
        command = packet[1]
        
        if command != 'PRIVMSG':
            print(data.decode())
        else:
            chat = packet[3][1:].replace('\r\n', '')
            print('*** [CHAT] %s: %s' % (nickname, chat))

    def connection_lost(self, exc):
        print('The server closed the connection')
        print('Stop the event loop')
        self.loop.stop()
        
    # call by main()
    def send_chat(self, chat):
        msg = 'PRIVMSG %s :%s\r\n' % (self.channel, chat)
        print('*** [SENDED]', msg)
        self.transport.write(msg.encode())

def main():
    host, port = parse_url(SERVER)
    assert (PASS != 'oauth:token'), 'you must set PASS'
    print('*** [SETTING_INFO] %s %d %s %s %s' % (host, port, CHANNEL, NICK, PASS))
    
    loop = asyncio.get_event_loop()
    if loop.is_closed():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
    client = IRCClient(CHANNEL, NICK, PASS, loop)
    coroutine = loop.create_connection(lambda: client, host, port)

    try:
        loop.run_until_complete(coroutine) # connect

        # send chatting
        client.send_chat('hello I am a bot')
        
        loop.run_forever() # data receive
    except KeyboardInterrupt as e:
        print(e)
    finally:
        loop.close()
    
if __name__ == '__main__':
    main()

*** [SETTING_INFO] chat.freenode.net 6666 #hojong hjbot_02371 None
*** [SENDED] NICK hjbot_02371
*** [SENDED] USER dummy 0 * :dummy
*** [SENDED] JOIN #hojong
:leguin.freenode.net NOTICE * :*** Looking up your hostname...
:leguin.freenode.net NOTICE * :*** Checking Ident
:leguin.freenode.net NOTICE * :*** Couldn't look up your hostname
:leguin.freenode.net NOTICE * :*** No Ident response
:leguin.freenode.net 001 hjbot_02371 :Welcome to the freenode Internet Relay Chat Network hjbot_02371
:leguin.freenode.net 002 hjbot_02371 :Your host is leguin.freenode.net[130.239.18.119/6666], running version ircd-seven-1.1.4
:leguin.freenode.net 003 hjbot_02371 :This server was created Tue Jan 3 2017 at 13:35:13 UTC
:leguin.freenode.net 004 hjbot_02371 leguin.freenode.net ircd-seven-1.1.4 DOQRSZaghilopswz CFILMPQSbcefgijklmnopqrstvz bkloveqjfI
:leguin.freenode.net 005 hjbot_02371 CHANTYPES=# EXCEPTS INVEX CHANMODES=eIbq,k,flj,CFLMPQScgimnprstz CHANLIMIT=#:120 PREFIX=(ov)@+ MAXLIST=bqeI:100 MODES=4 NE